## Uploading code

In this section you will learn how to create a working WiFi weather station built using an Adafruit Feather Huzzah ESP9266, and a temperature sensor. The working WiFi weather station will post the temperature to [ThingSpeak.com](https://thingspeak.com/)

Before the Micropython REPL (the Python prompt) running on the Adafruit Feather Huzzah ESP8266 can be used, Micropython needs to be installed on the board. Putty also needs to be installed on a computer in order for the computer to communicate with Feather Huzzah over serial. See the previous section on how to install Micropython on an Adafruit Feather Huzzah ESP9266 and how to install Putty on a Windows computer.

### Summary of Steps

1. Install **ampy** with **pip**
2. Write python code 
3. Put the code on the board with **ampy**
4. Run functions from the Micropyton REPL
5. Run a program

### Install ampy with pip

**Ampy** is a Python package developed by Adafruit, the company that makes the Feather Huzzah board. **Ampy** is used to push code stored on a computer to the Feather Huzzah board. **Ampy** can be installed using the **Anaconda Prompt**. Alternatively, **pip** can be used to install **ampy**. If using a virtual environment, active the virtual environment first before proceeding with the **ampy** package installation.

```text
> conda activate micropython
(micropython) > pip install ampy-adafruit
(micropython) > ampy --help
```

### Write Python Code

Now write the Python code that will be put on the board. The Feather Huzzah board has two main Python files: **boot.py** and **main.py**. Additional files can also be added to the board. **boot.py** is the file that runs first when the board is powered up. After **boot.py** runs, then **main.py** will run. Another **.py** file can be added to the board to provide **main.py** with some functions and classes to work with. 

Two general things need to be accomplished on the Feather Huzzah board to turn it into a WiFi weather station: read the temperature and post the temperature to ThingSpeak.com. Two different **.py** files will be constructed, one **.py** file for each of these general functionalities (reading the temperature and posting the temperature). 

The first **MCP9808.py** file will simplify reading temperature data off of the Adafruit MCP9808 temperature breakout. A function that parses out the temperature data from the I2C bus and return it as the output for the **readtemp** function will be created. The function needs to import the ```machine``` module to use the I2C bus. The machine module provides the class to create a new i2c object. When the i2c object is instantiated, the ```scl``` and ```sda``` pins that the sensor is connected to need to be specified. ```scl``` is the i2c clock line and ```sda``` is the i2c data line. These are pins 5 and 4 on the Adafruit Feather Huzzah. Then a new byte array variable needs to be created, so that later data from the sensor can be saved into it.  Next read the sensor data using the ```i2c.readfrom_mem_into()``` function. The first argument is the I2C bus address for the sensor. In this case the sensor is at I2C bus address ```24```. The line ```>>> i2c.scan()``` typed into the micropython REPL will show the I2C bus address.  The next function argument is the register on the MCP9808 temperature sensor where the temperature value is stored, which happens to be register ```5```. If register ```5``` is accessed on the MCP, the temperature can be recorded. The third arguments is the variable to store the temperature data into. The ```i2c.readfrom_mem_into()``` method changes the variable that is a method argument, rather than changing a variable which is the method output as most methods do. This is why the ```byte_data``` variable needs to be created before calling the ```i2c.readfrom_mem_into()``` method. Next some post processing is needed to convert the byte array into a temperature in degrees C.

```python
# MCP9808.py

# Functions for the  MCP9808 temperature sensor
# learn.adafruit.com/micropython-hardware-i2c-devices/i2c-master

def readtemp():
    import machine
    i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
    byte_data = bytearray(2)
    i2c.readfrom_mem_into(24, 5, byte_data)
    value = byte_data[0] << 8 | byte_data[1]
    temp = (value & 0xFFF) / 16.0
    if value & 0x1000:
        temp -= 256.0
    return temp
```

Now build a Python file for the set of WiFi functions. 

```python
#wifitools.py

def connect(SSID,password):
    import network
    sta_if = network.WLAN(network.STA_IF)
    if not sta_if.isconnected():
        print('connecting to network...')
        sta_if.active(True)
        sta_if.connect(SSID, password)
        while not sta_if.isconnected():
            pass
    print('network config:', sta_if.ifconfig())

        
def http_get(url):
    import socket
    _, _, host, path = url.split('/', 3)
    addr = socket.getaddrinfo(host, 80)[0][-1]
    s = socket.socket()
    s.connect(addr)
    str_send = 'GET /%s HTTP/1.0\r\nHost: %s\r\n\r\n' % (path, host)
    s.send(bytes(str_send,'utf8'))
    while True:
        data = s.recv(100)
        if data:
            print(str(data, 'utf8'), end='')
        else:
            break

def thingspeak_post(API_key,data):
    if not isinstance(data, str):
        data = str(data)
    base_url = 'https://api.thingspeak.com/update?api_key='
    mid_url = '&field1='
    url = base_url + API_key + mid_url + data
    http_get(url)

```

Now we will construact a Micropython script called **_main.py_** which will use the functions stored in **_MCP9808.py_**. The **_main.py_** script will import the **_MCP9808.py_** and **_wifitools_** functions then use the ```wifitools.connect()``` function to connect to the WiFi network. A ```time.sleep(5)``` line to allows time for the Feather Huzzah board to connect to the WiFi network. Next we'll run a loop for a total of 8 hours at 60 minutes each hour. Inside the loop, we'll read in the temperature from the MCP9808 using the ```MCP9808.readtemp()``` function and post it to ThingSpeak.com using the ```wifitools.thingspeak_post()``` function. To take the temperature once a minute, we need to ```time.sleep(60)``` (wait 60 seconds) between each measurement.

```python
# main.py
# ESP8266 Feather Huzzah Weather Station

import wifitools
import MCP9808
import time
import config

api_key = config.API_KEY
ssid = config.SSID
password = config.WIFI_PASSWORD

wifitools.connect(ssid,password)
time.sleep(5)

for i in range(8*60):
    data = MCP9808.readtemp()
    wifitools.thingspeak_post(api_key,data)
    time.sleep(60)
```

### Upload Python Code to the Feather Huzzah with **ampy**

Once all the **_.py_** files are created, ensure the Feather Huzzah board is connected with a USB cable, and be aware of which serial port the Feather Huzzah is connected to. Upload the code files to the board using **ampy**. Make sure you are in the directory with the **_.py_** files and that you are working in a virtual environment that has ```ampy``` installed in it. In the example code below, the ```(micropython)``` virtual environment is active.

```text
(micropython) > ampy --port COM4 put MCP9808.py
(micropython) > ampy --port COM4 put wifitools.py
(micropython) > ampy --port COM4 put main.py
(micropython) > ampy --port COM4 ls
boot.py
wifitools.py
MCP9808.py
config.py
main.py
```

### Unplug and power up the Feather Huzzah and watch the data on ThingSpeak.com

The Feather Huzzah needs to be restarted to run the code uploaded with **ampy**. To restart the board, unplug and then replug in the board's power (the USB cable). Once power is restored, the board will run through the **_boot.py_** script then start the **_main.py_** script. When the board runs the **_main.py_** script, the board will will connect to the WiFi network, read the temperature then upload the temperature to ThingSpeak.com. When ThingSpeak.com is viewed, you will see the temperature plotted on the Thinkspeak.com Channel's page.